In [ ]:
[23년도] '성별'과 '국민연금 납부여부' 항목 결측치 확인 및 대체

In [ ]:
welfare['sex'].value_counts()

In [ ]:
# 결측확인
welfare['sex'].isna().sum()

In [ ]:
# 성별, 항목 이름 부여
welfare['sex']=np.where(welfare['sex']==1,'male','female')

# 빈도 구하기
welfare['sex'].value_counts() # 빈도수

In [ ]:
welfare['pension_status'].value_counts('1')

In [ ]:
welfare['pension_status'].isna().sum()

In [ ]:
# 결측치의 개수 계산
missing_count = welfare['pension_status'].isna().sum()

# 각 값의 개수 계산
count_1 = int(missing_count * 0.8)  # 80%는 1로
count_2 = missing_count - count_1   # 나머지 20%는 2로

# 결측치를 채울 값 생성
fill_values = [1] * count_1 + [2] * count_2

# 결측치 위치의 인덱스 찾기
nan_indices = welfare[welfare['pension_status'].isna()].index

# 랜덤하게 결측치 인덱스를 선택하여 채우기
np.random.shuffle(fill_values)
welfare.loc[nan_indices, 'pension_status'] = fill_values

# 결과 확인
print("Pension Status Distribution After Filling:")
print(welfare['pension_status'].value_counts())

In [ ]:
# 결측치 개수 0 확인
welfare['pension_status'].isna().sum()

In [ ]:
# 국민연금 납부 여부 항목 이름 부여
welfare['pension_status']=np.where(welfare['pension_status']==1,'paid','unpaid')

[23년도] 성별에 따른 국민연금 납부 여부

In [ ]:
# 데이터 그룹화 및 집계
grouped_data = welfare.groupby(['sex', 'pension_status']).size().unstack()

# 막대 그래프 생성
grouped_data.plot(kind='bar', stacked=True, colormap='Set2')
plt.title('Pension Status by Sex')
plt.xlabel('Sex')
plt.ylabel('Count')
plt.legend(title='Pension Status')
plt.xticks(ticks=range(len(grouped_data.index)), labels=['Male', 'Female'], rotation=0)
plt.show()

In [ ]:
[23년도] '장애종류'와 '외래진료 횟수' 항목 결측치 확인 및 대체 및 제거

In [ ]:
welfare['disabled_type'].isna().sum()

In [ ]:
welfare['disabled_type'].value_counts()

In [ ]:
# 0:비장애인, 1:지체장애, 2:뇌병변장애, 3:시각장애, 4:청각장애 5~: 그 외

welfare['disabled']=np.where(welfare['disabled_type']==0, '비장애인',
                           np.where(welfare['disabled_type']==1,'지체장애',
                                    np.where(welfare['disabled_type']==2,'뇌병변장애',
                                             np.where(welfare['disabled_type']==3,'시각장애',
                                                      np.where(welfare['disabled_type']==4,'청각장애','그 외')))))

welfare['disabled'].head(10)

In [ ]:
welfare['hospital_year'].isna().sum()

In [ ]:
# 외래진료횟수 150회 이상은 결측치로 처리
welfare['hospital_year']=np.where(welfare['hospital_year']>=150,np.nan,welfare['hospital_year'])

In [ ]:
# 결측치 제거
welfare = welfare.dropna(subset=['hospital_year'])

[23년도] 장애종류에 따른 평균 외래진료 횟수

In [ ]:
# 'disabled' 열의 각 범주별로 'hospital_year' 평균 계산
average_hospital_visits = welfare.groupby('disabled')['hospital_year'].mean().reset_index()

# 막대 그래프 생성
plt.figure(figsize=(10, 8))
sns.barplot(data=average_hospital_visits, x='disabled', y='hospital_year', palette='viridis', order=['비장애인','지체장애','뇌병변장애','시각장애','청각장애','그 외'])
plt.title('장애종류에 따른 평균 외래진료 횟수')
plt.xlabel('장애종류')
plt.ylabel('평균 외래진료 횟수')
plt.xticks(rotation=0)  # x축 레이블을 읽기 쉽게 회전
plt.show()

[23년도] '월급' 항목 결측치 확인 및 대체 및 제거

In [ ]:
# 이상치는 결측 처리
welfare['income']=np.where(welfare['income']>=9500, np.nan, welfare['income'])

# 결측치 확인
welfare['income'].isna().sum()

In [ ]:
# 장애종류별 월급 평균
dis_income=welfare.dropna(subset=['income'])\
                  .groupby('disabled',as_index=False)\
                  .agg(mean_income=('income','mean'))
dis_income

[23년도] 장애종류별 평균 월급

In [ ]:
# 'disabled' 컬럼을 범주형으로 변환하고 순서 지정
dis_income['disabled'] = pd.Categorical(dis_income['disabled'], 
                                       categories=['비장애인', '지체장애', '뇌병변장애', '시각장애', '청각장애', '그 외'],
                                       ordered=True)

# 데이터 정렬
dis_income = dis_income.sort_values('disabled')


# 선 그래프 그리기
sns.lineplot(data=dis_income, x='disabled', y='mean_income', marker='h', color='red')
plt.title("장애종류별 평균 월급")
plt.xlabel('장애종류')
plt.ylabel('평균수입')
plt.xticks(rotation=0) # x축 범주들 글자 각도 조정, 필요에 따라 조절
plt.show()

[23년도] 장애인 취업률 구하기

In [ ]:
welfare['eW'].isna().sum()

In [ ]:
welfare = welfare.dropna(subset=['eW'])

In [ ]:
welfare['eW'].value_counts()

In [ ]:
welfare['eW'] = np.where((welfare['eW'] == 8) | (welfare['eW'] == 9), '비근로자', '근로자')

In [ ]:
welfare['eW'].value_counts('1')

In [ ]:
# 장애 종류의 순서를 지정하여 데이터 정렬
order = ['비장애인', '지체장애', '뇌병변장애', '시각장애', '청각장애', '그 외']
welfare['disabled'] = pd.Categorical(welfare['disabled'], categories=order, ordered=True)

# 데이터 그룹화 및 집계
grouped_data = welfare.groupby(['disabled', 'eW']).size().unstack().reindex(columns=['근로자', '비근로자'], fill_value=0)

# 막대 그래프 생성
plt.figure(figsize=(12, 8))
ax = grouped_data.plot(kind='bar', stacked=True, colormap='Set1', ax=plt.gca())

# 제목 및 축 레이블 설정
plt.title('Disability Type and Employment Status')
plt.xlabel('Disability Type')
plt.ylabel('Count')

# x축 레이블 설정
plt.xticks(ticks=range(len(grouped_data.index)), labels=grouped_data.index, rotation=45)

# 범례 설정
plt.legend(title='Employment Status')

# 그래프 표시
plt.show()

In [ ]:

# 장애 종류의 순서를 지정하여 데이터 정렬
order = ['비장애인', '지체장애', '뇌병변장애', '시각장애', '청각장애', '그 외']
welfare['disabled'] = pd.Categorical(welfare['disabled'], categories=order, ordered=True)

# 데이터 그룹화 및 집계
grouped_data = welfare.groupby(['disabled', 'eW']).size().unstack().reindex(columns=['근로자', '비근로자'], fill_value=0)

# 비장애인 제외
filtered_data = grouped_data.drop(index='비장애인')

# 비장애인을 제외한 나머지 범주의 총 수
total_counts_excluding_non_disabled = filtered_data.sum().sum()

# 비장애인을 제외한 나머지 범주에서 근로자의 총 수
total_employed_excluding_non_disabled = filtered_data['근로자'].sum()

# 비율 계산
employment_percentage = (total_employed_excluding_non_disabled / total_counts_excluding_non_disabled) * 100

# 결과 출력
print(f'장애인 중에서 근로자 비율: {employment_percentage:.2f}%')
print(f'장애인 중에서 비근로자 비율: {100-employment_percentage:.2f}%')